# Lightning Channel via Batch Transaction

In [ ]:
%load_ext autoreload
%autoreload 2

## Setup LND GRPC

In [ ]:
import sys

In [ ]:
# LND GRPC bindings should be in '/home/jovyan/lnrpc'
sys.path.append('/home/jovyan/lnrpc')

In [ ]:
%%sh
sudo cp /ali-lnd/data/chain/bitcoin/regtest/admin.macaroon /tmp/ali.macaroon
sudo chmod +r /tmp/ali.macaroon

sudo cp /bob-lnd/data/chain/bitcoin/regtest/admin.macaroon /tmp/bob.macaroon
sudo chmod +r /tmp/bob.macaroon

In [ ]:
from lnd_rpc import LndRpc, lnmsg, walletmsg

In [ ]:
ali = LndRpc(host='ali-lnd:10009',
             cert_path='/ali-lnd/tls.cert',
             macaroon_path='/tmp/ali.macaroon')

In [ ]:
bob = LndRpc(host='bob-lnd:10009',
             cert_path='/bob-lnd/tls.cert',
             macaroon_path='/tmp/bob.macaroon')

In [ ]:
aliNodePubKey = ali.lnd.GetInfo(lnmsg.GetInfoRequest()).identity_pubkey
aliNodePubKey

In [ ]:
bobNodePubKey = bob.lnd.GetInfo(lnmsg.GetInfoRequest()).identity_pubkey
bobNodePubKey

## Fund Ali and Bob

In [ ]:
import os
import bitcoin
from rpc import Proxy, Config

In [ ]:
bitcoin.SelectParams('regtest')

In [ ]:
brpc = Proxy(config=Config(
    rpcuser=os.environ['BTCD_RPCUSER'],
    rpcpassword=os.environ['BTCD_RPCPASS'],
    rpcconnect='bitcoind',
    rpcport=18443
))

In [ ]:
brpc.getblockcount()

In [ ]:
aliAddr = ali.lnd.NewAddress(lnmsg.NewAddressRequest(type=0)) # p2wkh
aliAddr

In [ ]:
bobAddr = bob.lnd.NewAddress(lnmsg.NewAddressRequest(type=0)) # p2wkh
bobAddr

In [ ]:
# fund ali
_ = list(brpc.generatetoaddress(1, aliAddr.address))

In [ ]:
# fund bob
_ = list(brpc.generatetoaddress(1, bobAddr.address))

In [ ]:
brpc.createwallet('miner')
minerAddr = brpc.getnewaddress("coinbase")
brpc.unloadwallet('miner')

In [ ]:
# unlock mined coins
_ = list(brpc.generatetoaddress(110, minerAddr))

In [ ]:
ali.lnd.WalletBalance(lnmsg.WalletBalanceRequest())

In [ ]:
bob.lnd.WalletBalance(lnmsg.WalletBalanceRequest())

## Create bid and ask

In [ ]:
import base64
import bitcoin.core as bc
import bitcoin.core.script as bs
from bitcoin.rpc import hexlify, unhexlify

### Ali taker

In [ ]:
# taker needs to pay premium to open channel
premiumAmount = 1 * bc.COIN # premium Ali is willing to pay
fundAmount = 10 * bc.COIN # requested inbound capacity

In [ ]:
# create dummy psbt to extract 'funding' UTXO and change addresses
# user minerAddr as a dummy address to create psbt
tx_template = walletmsg.TxTemplate(outputs={str(minerAddr): premiumAmount})

psbtRequest = walletmsg.FundPsbtRequest(raw=tx_template, target_conf=6)
psbtRequest

In [ ]:
# select funding UTXO and change output
# We create dummy p
psbt = ali.wallet.FundPsbt(request=psbtRequest)
psbt

In [ ]:
psbtBase64 = base64.b64encode(psbt.funded_psbt).decode()

In [ ]:
brpc._proxy._call('analyzepsbt', psbtBase64)

In [ ]:
psbtDecoded = brpc._proxy._call('decodepsbt', psbtBase64)
psbtDecoded['tx']

In [ ]:
aliInputs = []
for vin in psbtDecoded['tx']['vin']:
    txin = bc.CTxIn(bc.COutPoint(bc.lx(vin['txid']), vin['vout']))
    aliInputs.append(txin)
aliInputs

In [ ]:
# assume there are only 2 outputs
assert len(psbtDecoded['tx']['vout']) == 2

In [ ]:
aliChangeOutput = psbtDecoded['tx']['vout'][psbt.change_output_index]
aliChangeOutput = bc.CTxOut(int(aliChangeOutput['value'] * bc.COIN),
                            bc.CScript(unhexlify(aliChangeOutput['scriptPubKey']['hex'])))

aliChangeOutput

In [ ]:
key_family = 0 # multisig?
keyReq = walletmsg.KeyReq(key_family=key_family)
aliKeyDesc = ali.wallet.DeriveNextKey(keyReq)
aliKeyDesc = ali.wallet.DeriveNextKey(keyReq) # need to call twice?
aliKeyDesc

In [ ]:
takeRequest = {
    'nodeAddr': 'ali-lnd',
    'nodePubKey': aliNodePubKey,
    'premiumAmount': premiumAmount,
    'fundAmount': fundAmount,
    'inputs': aliInputs,
    'change': aliChangeOutput,
    'multisigKey': aliKeyDesc
}
takeRequest

### Bob maker

In [ ]:
premiumAmount = 1 * bc.COIN # premium Bob is willing to accept
fundAmount = 10 * bc.COIN # funding amount Bob is willing to provide

In [ ]:
# create dummy psbt to extract 'funding' UTXO and change addresses
# user minerAddr as a dummy address to create psbt
tx_template = walletmsg.TxTemplate(outputs={str(minerAddr): fundAmount})

psbtRequest = walletmsg.FundPsbtRequest(raw=tx_template, target_conf=6)
psbtRequest

In [ ]:
# select funding UTXO and change output
# We create dummy p
psbt = bob.wallet.FundPsbt(request=psbtRequest)
psbt

In [ ]:
psbtBase64 = base64.b64encode(psbt.funded_psbt).decode()

In [ ]:
psbtDecoded = brpc._proxy._call('decodepsbt', psbtBase64)
psbtDecoded['tx']

In [ ]:
bobInputs = []
for vin in psbtDecoded['tx']['vin']:
    txin = bc.CTxIn(bc.COutPoint(bc.lx(vin['txid']), vin['vout']))
    bobInputs.append(txin)
bobInputs

In [ ]:
# assume there are only 2 outputs
assert len(psbtDecoded['tx']['vout']) == 2

In [ ]:
bobChangeOutput = psbtDecoded['tx']['vout'][psbt.change_output_index]
bobChangeOutput = bc.CTxOut(int(bobChangeOutput['value'] * bc.COIN),
                            bc.CScript(unhexlify(bobChangeOutput['scriptPubKey']['hex'])))

bobChangeOutput

In [ ]:
key_family = 0 # multisig?
keyReq = walletmsg.KeyReq(key_family=key_family)
bobKeyDesc = bob.wallet.DeriveNextKey(keyReq)
bobKeyDesc = bob.wallet.DeriveNextKey(keyReq) # need to call twice?
bobKeyDesc

In [ ]:
makeRequest = {
    'nodeAddr': 'bob-lnd',
    'nodePubKey': bobNodePubKey,
    'premiumAmount': premiumAmount,
    'fundAmount': fundAmount,
    'inputs': bobInputs,
    'change': bobChangeOutput,
    'multisigKey': bobKeyDesc
}
makeRequest

## Auctioneer creates batch funding transaction

In [ ]:
takerPubKey = takeRequest['multisigKey'].raw_key_bytes
makerPubKey = makeRequest['multisigKey'].raw_key_bytes

In [ ]:
# https://github.com/lightningnetwork/lightning-rfc/blob/master/03-transactions.md#funding-transaction-output
if makerPubKey > takerPubKey:
    pk1, pk2 = takerPubKey, makerPubKey
else:
    pk1, pk2 = makerPubKey, takerPubKey

In [ ]:
msigScript = bs.CScript([
        bs.OP_2,
        pk1,
        pk2,
        bs.OP_2,
        bs.OP_CHECKMULTISIG
    ])

In [ ]:
fundingOutput = bc.CTxOut(takeRequest['premiumAmount'] + makeRequest['fundAmount'],
                          msigScript.to_p2sh_scriptPubKey())
fundingOutput

In [ ]:
batchInputs = takeRequest['inputs'] + makeRequest['inputs']
batchOutputs = [takeRequest['change'], makeRequest['change'], fundingOutput]

In [ ]:
batchTx = bc.CTransaction(batchInputs, batchOutputs)
batchTx

In [ ]:
batchTx.GetTxid()